<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.2.1
# *Querying the International Space Station*

## The OpenNotify API

The OpenNotify API exposes a few attributes of the International Space Station (ISS) via a simple, authentication-free interface. The simplicity of this API precludes any need for a dedicated Python library. However, as with many APIs, it accepts requests according to HTTP standards and returns responses in JSON format, so the Python libraries request and json will make managing the I/O simpler still.

In [1]:
import requests
import json
from datetime import datetime, date, time

This request fetches the latest position of the international space station:

In [2]:
response = requests.get("http://api.open-notify.org/iss-now.json") 

Print the status code and text of the response:

In [3]:
#ANSWER
response.status_code

200

In [4]:
#ANSWER
print(response.text)

{"iss_position": {"latitude": "-44.2635", "longitude": "-119.4753"}, "message": "success", "timestamp": 1722246860}


We can use another API to request the current position of the ISS and the next few times at which it will be over a certain location. The latitude and longitude of Sydney are (-33.87, 151.21).

In [5]:
response = requests.get("https://api.g7vrd.co.uk/v1/satellite-passes/25544/-33.87/151.21.json?minelevation=0&hours=24")

Print the response header:

In [6]:
#ANSWER
print(response.headers)

{'Date': 'Mon, 29 Jul 2024 09:54:24 GMT', 'Server': 'Apache', 'Vary': 'Origin,Access-Control-Request-Method,Access-Control-Request-Headers', 'Access-Control-Allow-Origin': '*', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '0', 'Cache-Control': 'no-cache, no-store, max-age=0, must-revalidate', 'Pragma': 'no-cache', 'Expires': '0', 'X-Frame-Options': 'DENY', 'Content-Type': 'application/json', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked'}


Print the content of the response (the data that the server returned):

In [7]:
#ANSWER
print(response.content)

b'{"api_status":"ALPHA","request_timestamp":"2024-07-29T09:54:24.224201136Z","norad_id":25544,"satellite_name":"ISS","tle_last_retrieved":"2024-07-28T22:42:18.680915784Z","lat":-33.87,"lon":151.21,"hours":24,"min_elevation":0,"query_ms":16,"passes":[{"start":"2024-07-29T12:48:34.208Z","tca":"2024-07-29T12:52:04.208Z","end":"2024-07-29T12:55:24.208Z","aos_azimuth":18,"los_azimuth":97,"max_elevation":5.0},{"start":"2024-07-29T14:22:39.208Z","tca":"2024-07-29T14:28:09.208Z","end":"2024-07-29T14:33:39.208Z","aos_azimuth":316,"los_azimuth":132,"max_elevation":78.0},{"start":"2024-07-29T16:00:29.208Z","tca":"2024-07-29T16:04:59.208Z","end":"2024-07-29T16:09:59.208Z","aos_azimuth":267,"los_azimuth":148,"max_elevation":13.0},{"start":"2024-07-29T17:40:14.208Z","tca":"2024-07-29T17:43:14.208Z","end":"2024-07-29T17:46:24.208Z","aos_azimuth":222,"los_azimuth":153,"max_elevation":4.0},{"start":"2024-07-29T19:18:19.208Z","tca":"2024-07-29T19:21:49.208Z","end":"2024-07-29T19:25:14.208Z","aos_azimuth

Note that this is a Python byte string:

In [8]:
print(type(response.content))

<class 'bytes'>


Print just the "content-type" value from the header:

In [9]:
#ANSWER
print(response.headers['content-type'])

application/json


JSON was designed to be easy for computers to read, not for people. The `requests` library can decode the JSON byte string:

In [10]:
overheads = response.json()
print(overheads)

{'api_status': 'ALPHA', 'request_timestamp': '2024-07-29T09:54:24.224201136Z', 'norad_id': 25544, 'satellite_name': 'ISS', 'tle_last_retrieved': '2024-07-28T22:42:18.680915784Z', 'lat': -33.87, 'lon': 151.21, 'hours': 24, 'min_elevation': 0, 'query_ms': 16, 'passes': [{'start': '2024-07-29T12:48:34.208Z', 'tca': '2024-07-29T12:52:04.208Z', 'end': '2024-07-29T12:55:24.208Z', 'aos_azimuth': 18, 'los_azimuth': 97, 'max_elevation': 5.0}, {'start': '2024-07-29T14:22:39.208Z', 'tca': '2024-07-29T14:28:09.208Z', 'end': '2024-07-29T14:33:39.208Z', 'aos_azimuth': 316, 'los_azimuth': 132, 'max_elevation': 78.0}, {'start': '2024-07-29T16:00:29.208Z', 'tca': '2024-07-29T16:04:59.208Z', 'end': '2024-07-29T16:09:59.208Z', 'aos_azimuth': 267, 'los_azimuth': 148, 'max_elevation': 13.0}, {'start': '2024-07-29T17:40:14.208Z', 'tca': '2024-07-29T17:43:14.208Z', 'end': '2024-07-29T17:46:24.208Z', 'aos_azimuth': 222, 'los_azimuth': 153, 'max_elevation': 4.0}, {'start': '2024-07-29T19:18:19.208Z', 'tca': '2

What kind of object did this give us?

In [11]:
#ANSWER:
print(type(overheads))

<class 'dict'>


Python dicts are easier to work with, but the data we want is still buried in that data structure, so we have to dig it out. First, extract the `passes` value to a separate list:

In [12]:
#ANSWER:
passes = overheads['passes']

In [13]:
passes

[{'start': '2024-07-29T12:48:34.208Z',
  'tca': '2024-07-29T12:52:04.208Z',
  'end': '2024-07-29T12:55:24.208Z',
  'aos_azimuth': 18,
  'los_azimuth': 97,
  'max_elevation': 5.0},
 {'start': '2024-07-29T14:22:39.208Z',
  'tca': '2024-07-29T14:28:09.208Z',
  'end': '2024-07-29T14:33:39.208Z',
  'aos_azimuth': 316,
  'los_azimuth': 132,
  'max_elevation': 78.0},
 {'start': '2024-07-29T16:00:29.208Z',
  'tca': '2024-07-29T16:04:59.208Z',
  'end': '2024-07-29T16:09:59.208Z',
  'aos_azimuth': 267,
  'los_azimuth': 148,
  'max_elevation': 13.0},
 {'start': '2024-07-29T17:40:14.208Z',
  'tca': '2024-07-29T17:43:14.208Z',
  'end': '2024-07-29T17:46:24.208Z',
  'aos_azimuth': 222,
  'los_azimuth': 153,
  'max_elevation': 4.0},
 {'start': '2024-07-29T19:18:19.208Z',
  'tca': '2024-07-29T19:21:49.208Z',
  'end': '2024-07-29T19:25:14.208Z',
  'aos_azimuth': 205,
  'los_azimuth': 127,
  'max_elevation': 5.0},
 {'start': '2024-07-29T20:54:39.208Z',
  'tca': '2024-07-29T20:59:39.208Z',
  'end': '2024

Now extract the `start` strings into an array called `srisetimes`:

In [14]:
#ANSWER:
srisetimes = [xpass['start']for xpass in passes]

In [15]:
srisetimes

['2024-07-29T12:48:34.208Z',
 '2024-07-29T14:22:39.208Z',
 '2024-07-29T16:00:29.208Z',
 '2024-07-29T17:40:14.208Z',
 '2024-07-29T19:18:19.208Z',
 '2024-07-29T20:54:39.208Z',
 '2024-07-29T22:31:09.208Z',
 '2024-07-30T00:11:04.208Z',
 '2024-07-30T13:34:04.208Z']

These are strings. We convert these to an array of Python `datetime` values called `risetimes`:

In [16]:
risetimes = [datetime.strptime(xpass['start'], "%Y-%m-%dT%H:%M:%S.%fZ") for xpass in passes]
risetimes

[datetime.datetime(2024, 7, 29, 12, 48, 34, 208000),
 datetime.datetime(2024, 7, 29, 14, 22, 39, 208000),
 datetime.datetime(2024, 7, 29, 16, 0, 29, 208000),
 datetime.datetime(2024, 7, 29, 17, 40, 14, 208000),
 datetime.datetime(2024, 7, 29, 19, 18, 19, 208000),
 datetime.datetime(2024, 7, 29, 20, 54, 39, 208000),
 datetime.datetime(2024, 7, 29, 22, 31, 9, 208000),
 datetime.datetime(2024, 7, 30, 0, 11, 4, 208000),
 datetime.datetime(2024, 7, 30, 13, 34, 4, 208000)]

Finally, use `risetime.strftime` to print these in a format that people understand:

```
e.g.
18/10/22 07:05
18/10/22 08:41
18/10/22 10:20
18/10/22 12:00
18/10/22 01:37
18/10/22 03:13
```



In [17]:
#ANSWER:
for risetime in risetimes:
    print(risetime.strftime("%d-%m-%y %I:%M"))

29-07-24 12:48
29-07-24 02:22
29-07-24 04:00
29-07-24 05:40
29-07-24 07:18
29-07-24 08:54
29-07-24 10:31
30-07-24 12:11
30-07-24 01:34


Finally, here is an endpoint that tells us who is on board:

In [18]:
response = requests.get("http://api.open-notify.org/astros.json")

In [19]:
print(response)

<Response [200]>


Referring to the methods used above, extract the number of astronauts and their names:

In [20]:
#ANSWER:
response.content

b'{"people": [{"craft": "ISS", "name": "Oleg Kononenko"}, {"craft": "ISS", "name": "Nikolai Chub"}, {"craft": "ISS", "name": "Tracy Caldwell Dyson"}, {"craft": "ISS", "name": "Matthew Dominick"}, {"craft": "ISS", "name": "Michael Barratt"}, {"craft": "ISS", "name": "Jeanette Epps"}, {"craft": "ISS", "name": "Alexander Grebenkin"}, {"craft": "ISS", "name": "Butch Wilmore"}, {"craft": "ISS", "name": "Sunita Williams"}, {"craft": "Tiangong", "name": "Li Guangsu"}, {"craft": "Tiangong", "name": "Li Cong"}, {"craft": "Tiangong", "name": "Ye Guangfu"}], "number": 12, "message": "success"}'

In [21]:
response.text

'{"people": [{"craft": "ISS", "name": "Oleg Kononenko"}, {"craft": "ISS", "name": "Nikolai Chub"}, {"craft": "ISS", "name": "Tracy Caldwell Dyson"}, {"craft": "ISS", "name": "Matthew Dominick"}, {"craft": "ISS", "name": "Michael Barratt"}, {"craft": "ISS", "name": "Jeanette Epps"}, {"craft": "ISS", "name": "Alexander Grebenkin"}, {"craft": "ISS", "name": "Butch Wilmore"}, {"craft": "ISS", "name": "Sunita Williams"}, {"craft": "Tiangong", "name": "Li Guangsu"}, {"craft": "Tiangong", "name": "Li Cong"}, {"craft": "Tiangong", "name": "Ye Guangfu"}], "number": 12, "message": "success"}'

In [22]:
astros = response.json()
print(astros)
print(astros['number'])  
for astronaut in astros['people']: 
    print(astronaut['name'])

{'people': [{'craft': 'ISS', 'name': 'Oleg Kononenko'}, {'craft': 'ISS', 'name': 'Nikolai Chub'}, {'craft': 'ISS', 'name': 'Tracy Caldwell Dyson'}, {'craft': 'ISS', 'name': 'Matthew Dominick'}, {'craft': 'ISS', 'name': 'Michael Barratt'}, {'craft': 'ISS', 'name': 'Jeanette Epps'}, {'craft': 'ISS', 'name': 'Alexander Grebenkin'}, {'craft': 'ISS', 'name': 'Butch Wilmore'}, {'craft': 'ISS', 'name': 'Sunita Williams'}, {'craft': 'Tiangong', 'name': 'Li Guangsu'}, {'craft': 'Tiangong', 'name': 'Li Cong'}, {'craft': 'Tiangong', 'name': 'Ye Guangfu'}], 'number': 12, 'message': 'success'}
12
Oleg Kononenko
Nikolai Chub
Tracy Caldwell Dyson
Matthew Dominick
Michael Barratt
Jeanette Epps
Alexander Grebenkin
Butch Wilmore
Sunita Williams
Li Guangsu
Li Cong
Ye Guangfu


## HOMEWORK


1. Write a simple handler for the response status code (refer to lab resources slide for HTTP response codes). As this Jupyter Notebook is an interactive device, the handler does not need to manage subsequent code execution (i.e. by branching or aborting execution), although it should return something that could be used to do so if deployed in a Python program.

In [23]:
#ANSWER:
def handleResponse(response, verbose = False):
    '''
    Returns Boolean Value, Status Code,
    '''
  # if Status Code is 200 return false, and status code
  # Otherwise Return True and Status Code

2. Test your response handler on some correct and incorrect API calls.

In [26]:
response = requests.get("http://api.open-notify.org/astros.json")
if handleResponse(response):
    print('API call successful!')
else:
    print('API call failed. Resolve issue before continuing!')

response = requests.get("http://api.open-notify.org/iss-now.json")
handleResponse(response, True)

API call failed. Resolve issue before continuing!


>

>

>



---



---



> > > > > > > > > © 2024 Institute of Data


---



---



